# 15.4 Longest common subsequence
In biology, we can express a strand of DNA (the genetic material of our body) over the finite set of "bases" $\{A, C, G, T\}$. 

## Subsequence, common subsequence, longest common subsequence
A way to measure the similarity of strands $S_1$ and $S_2$ is by finding a third strand $S_3$ in which the bases in $S_3$ appear in each of $S_1$ and $S_2$. In addition, these bases must appear in the same order, but NOT necessarily consecutively. For example, if $X=\langle A,B,C,B,D,A,B\rangle$, and $Y=\langle B,D,C,A,B,A\rangle$:
* There are $2^m$ possible **subsequences** of $X$ and $2^n$ of $Y$, where $m,n$ are length of $X,Y$ respectively
* $\langle B,C,A\rangle$ and $\langle B,C,A,B\rangle$ are the **common subsequences** of $X,Y$
* $\langle B,C,A,B\rangle$ is the **longest common subsequences** (LCS) of $X,Y$, because it has the maximum length among all the common subsequences of $X,Y$

In the LCS problem, we are given to sequences $X=\langle x_1,x_2,...,x_m\rangle$ and $Y=\langle y_1,y_2,...y_m\rangle$ and wish to find a maximum-length common sequence (i.e. LCS) of them. This section shows how to efficiently solve the LCS problem using **dynamic programming**.

## Step 1: Characterizing a LCS
### The brute-force approach is impractical
If we wish to apply a brute-force approach, we would: 
1. Enumerate all subsequences of X (there are $2^m$ of them!)
2. Check each subsequence of X to see whether it is also a subseuqence of Y (there are $2^n$ of them!)
3. Keep track of the longest subsequence we find

The time complexity is thus exponential ($O(2^{m+n})$), making it impractical for long sequences.

### The LCS probelm has an optimal-substructure property
Let $X=\langle x_1,x_2,...,x_m\rangle$ and $Y=\langle y_1,y_2,...y_m\rangle$, and let $Z=\langle z_1,z_2,...,z_k\rangle$ be the LCS of $X,Y$, we can show that:
1. If $x_m=y_n$, then $z_k=x_m=y_n$ and $Z_{k-1}=\langle z_1,z_2,...,z_k\rangle$ is the LCS of $X_{m-1},Y_{n-1}$
2. If $x_m\neq y_n$, then $z_k\neq x_m$ implies that $Z$ is the LCS of $X_{m-1},Y$
3. If $x_m\neq y_n$, then $z_k\neq y_n$ implies that $Z$ is the LCS of $X,Y_{m-1}$

## Step 2: A recursive solution
### LCS has the overlapping-subproblems property
To find an LCS of $X,Y$, we may need to find the LCSs of $X,Y_{n-1}$ and of $X_{m-1},Y_{n-1}$. Observe that each of these subproblems involves finding an LCS of $X_{m-1},Y_{n-1}$, many other subproblens share subsubproblems. The total number of subproblems is $mn$.

### Recursive formula based on condition
The optimal substructure LCS problem implies that we should examine **either one or two subproblems** depending on the condition of $(x_m,y_n)$. 
* Case 1: If $x_m=y_n$, we must find an LCS of $X_{m-1},Y_{n-1}$
* Case 2: If $x_m\neq y_n$, we must find an LCSs of $X_{m-1},Y$ and $X,Y_{m-1}$ and take the longer one

If we define $c[i,j]$ to be the length of an LCS of the sequences $X_i,Y_j$, we have the recursive formula:

$$
\begin{align}
c[i,j]=\left\{
        \begin{array}{ll}
        0 &\text{if}\ i=0 \:\text{or}\ j=0\\
        c[i-1,j-1]+1 &\text{if}\ i,j>0 \:\text{and}\ x_i=y_j\\
        \max(c[i-1,j],c[i,j-1]) &\text{if}\ i,j>0 \:\text{and}\ x_i\neq y_j\\
        \end{array}       
        \right.
\end{align}
$$

$*$ In contrast to rod cutting and matrix-chain multiplication, we ruled out no subproblems based on conditions.

Procedure `lcs_length` is a bottom-up DP to calculate solutions to LCS. 
* It takes two sequences $X=\langle x_1,x_2,...,x_m\rangle$ and $Y=\langle y_1,y_2,...y_m\rangle$ as input
* It stores the $c[i,j]$ (i.e. length of LCSs) values in a table $c[0..m,0..n]$
    * $c[m,n]$ is the length of an LCS of $X,Y$
* It maintains the table $b[1..m,1..n]$ to help us construct an **optimal solution**
    * $b[i,j]$ points to the table entry corresponding to the optimal subproblem solution chosen when computing $c[i,j]$

In [1]:
import numpy as np

In [24]:
def lcs_length(X,Y):
    m=len(X)
    n=len(Y)
    X=np.concatenate(([''],X)) #so that X starts from X[1] instead of X[0]
    Y=np.concatenate(([''],Y)) #so that Y starts from Y[1] instead of Y[0]
    c=np.zeros((m+1,n+1))
    b=np.full((m+1,n+1),fill_value=None)
    for i in range(1,m+1):
        for j in range(1,n+1):
            if X[i]==Y[j]: #* Case 1: If $x_m=y_n$, we must find an LCS of $X_{m-1},Y_{n-1}$
                c[i,j]=c[i-1,j-1]+1
                b[i,j]='up-left'
            elif c[i-1,j]>=c[i,j-1]: # Case 2: If $x_m!= y_n$,and length of c[i-1,j]>=c[i,j-1]
                c[i,j]=c[i-1,j]      # we take LCS of c[i-1,j] as LCS of $Xi,Yj$
                b[i,j]='up'          
            else: # Case 2: If $x_m!= y_n$,and length of c[i-1,j]<c[i,j-1]
                c[i,j]=c[i,j-1] # we take LCS of c[i,j-1] as LCS of $Xi,Yj$
                b[i,j]='left'
    return c,b

In [25]:
Xseq=np.array(['A','B','C','B','D','A','B'])
Yseq=np.array(['B','D','C','A','B','A'])
lcs_length(Xseq,Yseq)

(array([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1.],
        [0., 1., 1., 1., 1., 2., 2.],
        [0., 1., 1., 2., 2., 2., 2.],
        [0., 1., 1., 2., 2., 3., 3.],
        [0., 1., 2., 2., 2., 3., 3.],
        [0., 1., 2., 2., 3., 3., 4.],
        [0., 1., 2., 2., 3., 4., 4.]]),
 array([[None, None, None, None, None, None, None],
        [None, 'up', 'up', 'up', 'up-left', 'left', 'up-left'],
        [None, 'up-left', 'left', 'left', 'up', 'up-left', 'left'],
        [None, 'up', 'up', 'up-left', 'left', 'up', 'up'],
        [None, 'up-left', 'up', 'up', 'up', 'up-left', 'left'],
        [None, 'up', 'up-left', 'up', 'up', 'up', 'up'],
        [None, 'up', 'up', 'up', 'up-left', 'up', 'up-left'],
        [None, 'up-left', 'up', 'up', 'up', 'up-left', 'up']], dtype=object))

## Step 4: Constructing an LCS
The $b$ table returned by `LCS_length` enables us to quickly construct an LCS of $X, Y$. We begin at $b[m,n]$ and trace through the table by following the arrows:
* An "up-left" in entry $b[m,n]$ means that $x_i=y_j$ is an element of the LCS found
* An "up" means we should move upward in $i-1$ direction
* An "left" means we should move left in $j-1$ direction.

With this method, we encounter the elements of this LCS in **reverse order**. The following recursive procedure `print_LCS`prints out an LCS of $X,Y$ in the forward order:

In [50]:
def print_lcs(b,X,i,j):
    if i==0 or j==0:
        return
    if b[i,j]=='up-left':
        print_lcs(b,X,i-1,j-1)
        print (X[i]) #Note: print after recursive call!!!
    elif b[i,j]=='up':
        print_lcs(b,X,i-1,j)
    else:
        print_lcs(b,X,i,j-1)

In [49]:
b=lcs_length(Xseq,Yseq)[1] # table b from lcs_length
i=len(Xseq) #length of X
j=len(Yseq) #length of Y
X=np.concatenate(([''],Xseq)) #so that X starts from X[1] instead of X[0]
print_lcs(b,X,i,j)

B
C
B
A
